# Generating Inputs of Y Variable
# Yahoo Finance API (yahoo_fin)

In [615]:
# Initial imports
import os
import requests # not in use for pulling stock price
import requests_html # not in use for pulling stock price
import pandas as pd
from pathlib import Path

from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

# Yahoo_fin api to pull adjusted close price
import yahoo_fin.stock_info as si
from datetime import date
import datetime, timedelta
from yahoo_earnings_calendar import YahooEarningsCalendar

# Alternative to pulling the data for earnings calls without yahoo earnings, which does not work
import json
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'timedelta'

In [607]:
# We use yahoo_fin API to pull adjusted closing prices. We tried Alpaca API, but priecs are not adjusted for stock splits
# We define the tickers and other inputs for the API
ticker_list = ['TSLA', 'QQQ']
start_date='2010-06-29'
end_date=date.today()#'2021-01-30'
index_as_date=True
interval='1d'


# we create an empty data frame where we will store the final data
df_closing_prices = pd.DataFrame()

# Yahoo fin stores data in dictionary. We will create a for loop to pull data for all the tickers we need. We will use the same loop to create the data frame
# We create an empty dictionary where we will store data from the API
historical_data = {}

for ticker in ticker_list:
    historical_data[ticker] = si.get_data(ticker,start_date,end_date,index_as_date,interval)
    df_closing_prices[ticker]=historical_data[ticker]['adjclose']

# Add a title to index
df_closing_prices.index.name = 'date'


In [554]:
date_from = datetime.datetime.strptime('Jun 29 2020  10:00AM', '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime('Jan 19 2021  10:00AM', '%b %d %Y %I:%M%p')
type(date_from)
date_to

datetime.datetime(2021, 1, 19, 10, 0)

In [614]:
# Funciton to pull earnings dates for any stock
# uses Beautiful Soup to scrape data from yahoo finance

def my_earnings_dates(symbol, start_date, end_date):

    #     Define the URL
    url = f"https://finance.yahoo.com/calendar/earnings?from={start_date}&to={end_date}&symbol={symbol}"

    #     prepare start and end date to pass if conditions inside funciton only
    start_date=datetime.datetime.strptime(str(start_date), '%Y-%m-%d')
    end_date=datetime.datetime.strptime(str(end_date), '%Y-%m-%d')
    
    #     Get the data from the URL and use beautiful soup to parse it 
    response_data = requests.get(url)
    soup = BeautifulSoup(response_data.text)
    
    #     Find all the earnings dates associated to the symbol and date
    date_zone=soup.find_all('td',attrs={'aria-label':'Earnings Date'})

    #     start populating the dates and time zones with a for loop
    date_list=[]
    time_zone_list=[]
    
    
    for i in range(0,len(all_data)):
        
        #     print(all_data[i].text)
        #     Pull the date and time for each earnings call
        date_time=date_zone[i].find_all('span')[0]
        date_time=date_time.text
        date_time=datetime.datetime.strptime(date_time, '%b %d, %Y, %I %p')
         
        if date_time>=start_date and date_time<=end_date:
            #     we convert the format into a time string
            date_time=date_time.strftime('%Y-%m-%d %I:%M%p')

            #     Pull the time zone
            time_zone=date_zone[i].find_all('span')[1]
            time_zone=time_zone.text

            #     append date_time and time_zone 
            date_list.append(date_time)
            time_zone_list.append(time_zone)
    
    
    #    Create data frame with output data
    earnings_dates=pd.DataFrame()
    earnings_dates['date']=date_list
    earnings_dates['time zone']=time_zone
    earnings_dates.sort_values(by=['date'], inplace=True)
    
    return earnings_dates.sort_values(by=['date'])

# Pull earnings dates
# Define ticker
# Start and end date are defined above
symbol= 'TSLA'
# call funciton
earnings_dates_df=my_earnings_dates(symbol, start_date, end_date)

# Make sure to adjust data if needed - adjustment by market hour 

# we change the name to date column - we will drop this field later. We need a date field that shows off market hour tweets as t+1 
earnings_dates_df.rename(columns={'date':'date original'},inplace=True)
earnings_dates_df['date original']=pd.to_datetime(earnings_dates_df['date original'])

# Make earnings calls after 4pm fall into the following day
# Define market hour limit as everything after 16hs 00 min 00 sec
min_hour=16
min_minute=0
min_second=0

# we create the new field equalt to date original 
earnings_dates_df['date']=earnings_dates_df['date original'].copy()

# we add 1 day to date original if the tweet occured off market hours
earnings_dates_df.loc[(earnings_dates_df['date original'].dt.hour>=min_hour) & (earnings_dates_df['date original'].dt.minute>min_minute) & (earnings_dates_df['date original'].dt.second>min_second), 'date'] = earnings_dates_df['date original']+timedelta(days=1)




NameError: name 'timedelta' is not defined

In [161]:

# url = 'https://finance.yahoo.com/calendar/earnings?symbol={0}'.format(symbol)
url = f"https://finance.yahoo.com/calendar/earnings?from={start_date}&to={end_date}&symbol={symbol}"
print(url)

https://finance.yahoo.com/calendar/earnings?from=2010-06-29&to=2021-01-19&symbol=TSLA


In [503]:
response_data = requests.get(url)
soup = BeautifulSoup(response_data.text)

date_zone=soup.find_all('td',attrs={'aria-label':'Earnings Date'})

date_list=[]
time_zone_list=[]
for i in range(0,len(all_data)):
#     print(all_data[i].text)
    #     Pull the date and time
    date_time=date_zone[i].find_all('span')[0]
    date_time=date_time.text
    
    #     we convert the format into a time string
    date_time=datetime.datetime.strptime(date_time, '%b %d, %Y, %I %p').strftime('%Y-%m-%d %I:%M%p')
        
    #     Pull the time zone
    time_zone=date_zone[i].find_all('span')[1]
    time_zone=time_zone.text

    #     append date_time and time_zone 
    date_list.append(date_time)
    time_zone_list.append(time_zone)

  
# a=date_time
# a='Nov 09, 2010, 6 PM'

# t=datetime.datetime.strptime(a, '%b %d, %Y, %I %p').strftime('%Y-%m-%d %I:%M%p')
# date.parse(a)                                    
# type(t)

date_list
# date_list.date()



['2021-10-19 05:00AM',
 '2021-07-20 05:00AM',
 '2021-04-27 05:00AM',
 '2021-01-27 04:00PM',
 '2020-10-21 12:00AM',
 '2020-07-22 12:00AM',
 '2020-04-29 12:00AM',
 '2020-01-29 12:00AM',
 '2019-10-23 12:00AM',
 '2019-07-24 12:00AM',
 '2019-04-24 12:00AM',
 '2019-01-30 12:00AM',
 '2018-10-24 12:00AM',
 '2018-08-01 12:00AM',
 '2018-05-02 12:00AM',
 '2018-02-07 12:00AM',
 '2017-11-01 12:00AM',
 '2017-08-02 12:00AM',
 '2017-05-03 12:00AM',
 '2017-02-22 12:00AM',
 '2016-10-26 12:00AM',
 '2016-08-03 12:00AM',
 '2016-05-04 12:00AM',
 '2016-02-10 12:00AM',
 '2015-11-03 12:00AM',
 '2015-08-05 12:00AM',
 '2015-05-06 12:00AM',
 '2015-02-11 12:00AM',
 '2014-11-05 12:00AM',
 '2014-07-31 12:00AM',
 '2014-05-07 12:00AM',
 '2014-02-19 12:00AM',
 '2013-11-05 12:00AM',
 '2013-08-07 12:00AM',
 '2013-05-08 12:00AM',
 '2013-02-20 12:00AM',
 '2012-11-05 12:00AM',
 '2012-07-25 12:00AM',
 '2012-05-09 12:00AM',
 '2012-02-15 12:00AM',
 '2011-11-02 12:00AM',
 '2011-08-03 12:00AM',
 '2011-05-04 12:00AM',
 '2011-02-1

In [325]:
l={}
u=list()
# print(all_data[0])
for i in range(0,len(all_data)):#len(all_data)):
#     print(i)
    
    try:
        all_data_td = all_data[i].find('span')
        print(all_data_td)
    except:
         all_data_td = None

#     l[all_data_td[0].text]= all_data_td[1].text
    u.append(l)
#     l={}
print(u)

<span data-reactid="31">Oct 19, 2021, 5 AM</span>
<span data-reactid="46">Jul 20, 2021, 5 AM</span>
<span data-reactid="61">Apr 27, 2021, 5 AM</span>
<span data-reactid="76">Jan 27, 2021, 4 PM</span>
<span data-reactid="91">Oct 21, 2020, 12 AM</span>
<span data-reactid="106">Jul 22, 2020, 12 AM</span>
<span data-reactid="121">Apr 29, 2020, 12 AM</span>
<span data-reactid="136">Jan 29, 2020, 12 AM</span>
<span data-reactid="151">Oct 23, 2019, 12 AM</span>
<span data-reactid="166">Jul 24, 2019, 12 AM</span>
<span data-reactid="181">Apr 24, 2019, 12 AM</span>
<span data-reactid="196">Jan 30, 2019, 12 AM</span>
<span data-reactid="211">Oct 24, 2018, 12 AM</span>
<span data-reactid="226">Aug 01, 2018, 12 AM</span>
<span data-reactid="241">May 02, 2018, 12 AM</span>
<span data-reactid="256">Feb 07, 2018, 12 AM</span>
<span data-reactid="271">Nov 01, 2017, 12 AM</span>
<span data-reactid="286">Aug 02, 2017, 12 AM</span>
<span data-reactid="301">May 03, 2017, 12 AM</span>
<span data-reactid="3

In [243]:
u

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},


In [259]:
l={}
u=list()
# print(all_data[0])
for i in range(1,len(all_data)):
#     print(i)
    try:
        all_data_td = all_data[i].find_all('td')
    except:
        all_data_td = None
    
    l[all_data_td[0].text] = all_data_td[1].text
    u.append(l)
    l={}


IndexError: list index out of range

In [232]:
l

{}

In [85]:
# url='https://finance.yahoo.com'
# url='https://finance.yahoo.com/calendar/earnings'
# url = url + "?format=json"
requests.get(url)
# print(url)
# Execute GET request and store response

# Get content
response_content = response_data.content
# print(response_content)

# Formatting as json
# data = response_data.json()
# print(json.dumps(data, indent=4))

In [149]:
# response_content

In [3]:
df_closing_prices.head()

,TSLA,QQQ
date,,
2010-06-29,4.778,39.031284
2010-06-30,4.766,38.437302
2010-07-01,4.392,38.329292
2010-07-02,3.840,38.221321
2010-07-06,3.222,38.338306


In [51]:
# Load .env environment variables
load_dotenv()

True

In [53]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [54]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [55]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [57]:
# Format current date as ISO format
today = pd.Timestamp("2021-01-14", tz="America/New_York").isoformat()

# Format start and end dates as ISO format for one year period
start = pd.Timestamp("2010-06-29", tz="America/New_York").isoformat()

end = today

In [60]:
# Get closing prices for TWTR and QQQ from the last year


get_calendar(self, start, end)
                
                
# Display sample data
df_portfolio_year.head(10)

NameError: name 'get_calendar' is not defined

In [45]:
help('alpaca_trade_api.rest')

Help on module alpaca_trade_api.rest in alpaca_trade_api:

NAME
    alpaca_trade_api.rest

CLASSES
    builtins.Exception(builtins.BaseException)
        APIError
        RetryException
    builtins.object
        REST
    
    class APIError(builtins.Exception)
     |  APIError(error, http_error=None)
     |  
     |  Represent API related error.
     |  error.status_code will have http status code.
     |  
     |  Method resolution order:
     |      APIError
     |      builtins.Exception
     |      builtins.BaseException
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, error, http_error=None)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  ----------------------------------------------------------------------
     |  Readonly properties defined here:
     |  
     |  code
     |  
     |  request
     |  
     |  response
     |  
     |  status_code
     |  
     |  --------------------

In [4]:
# Export csv files with stock prices
file_name="stock_price.csv"
output_file = Path(f"../Resources/{file_name}")
df_closing_prices.to_csv(f"{output_file}")